In [1]:
!pip -q install gensim fuzzywuzzy python-Levenshtein

     |████████████████████████████████| 51kB 2.5MB/s 


In [2]:
!wget -qO "training data.xlsx" https://cdn.skillenza.com/files/6a5fa354-63f4-4075-ae9f-ed47b60c41c7/Train-Data.xlsx

In [24]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.decomposition import TruncatedSVD
from itertools import chain
from collections import Counter
import regex as re
import pickle
import gensim
from fuzzywuzzy import process

In [10]:
t = pd.read_excel("training data.xlsx")

In [11]:
t["wt"] = np.asarray([int(float(re.sub("\D", "", str(x)))) if re.sub("\D", "", str(x)) != "" else -99999 for x in t['Grammage'].values], dtype=int)

In [12]:
t.to_pickle("train.pkl")

In [13]:
def preproc(s):
  return [x for x in gensim.utils.simple_preprocess(s) if x not in {'gm', 'ml', 'kg', 'with', 'for', 'the', 'rs', 'of', 'under', 'less', 'more', 'than', 'lower', 'greater'}]

In [14]:
desc = t["Product Description"].tolist()
dlist = []
for i, des in enumerate(desc):
    dlist.append(preproc(des))
with open('dlist.pkl', 'wb') as f:
    pickle.dump(dlist, f)

In [15]:
model = Word2Vec(dlist, size=50, window=15, min_count=1, workers=8, iter=50)
model.save("word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [16]:
frequencies = Counter(list(chain.from_iterable(dlist)))
with open('freq.pkl', 'wb') as f:
    pickle.dump(frequencies, f)

In [64]:

def run_sif(query, sentences2, model, freqs={}, a=0.001, fuzz=3): 
    total_freq = sum(freqs.values())
    embeddings = []
    print(query)
    tokens1 = []
    for token in query:
        if token in model.wv:
            tokens1.append(token)
        else:
            for i in process.extract(token, model.wv.vocab.keys(), limit=fuzz):
                tokens1.append(i[0])
    print(tokens1)         
    if not tokens1: return None
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    embedding1 = np.zeros((len(sentences2), model.trainables.layer1_size)) + np.average([model.wv[token] for token in tokens1], axis=0, weights=weights1)
    
    embedding2 = np.zeros((len(sentences2), model.trainables.layer1_size))

      # SIF requires us to first collect all sentence embeddings and then perform 
      # common component analysis.
    for i, sent2 in enumerate(sentences2): 
          
          tokens2 = [token for token in sent2 if token in model.wv]             
          n = len(set(tokens1) & set(tokens2))/len(tokens1)
          
          weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]          
          embedding2[i] = np.average([model.wv[token] for token in tokens2], axis=0, weights=weights2)                  
          embedding1[i] += 15*n*embedding2[i]

    sims = np.einsum('ij,ij->i', embedding1, embedding2)/(np.linalg.norm(embedding1, axis=1)*np.linalg.norm(embedding2, axis=1))

    return sims

In [18]:
def wt(q):
    w = i = None
    for i, x in enumerate(q):
        if x in ('gm', 'ml', 'kg', 'g', 'l', 'lt', 'ltr', 'ml', 'pcs', 'xgm', ):
          if i:
              try:
                  w = int(float(q[i-1]))
              except: pass
    return w

In [19]:
def cost(q):
    more = c = i = None
    for i, x in enumerate(q):
        if x in ('less', 'lower'):
            if i < len(q)-2:
                try:
                    c = int(float(q[i+2]))
                    more = 1
                except: pass
        elif x == "under":
            if i < len(q)-1:
                try:
                    c = int(float(q[i+1]))
                    more = 1
                except: pass
        elif x in ('more', 'greater'):
            if i < len(q)-2:
                try:
                    c = int(float(q[i+2]))
                    more = 3
                except: pass
        if x == 'rs':
            if i:
                try:
                    c = int(float(q[i-1]))
                    more = 2
                except: pass
            else:
                try:
                    c = int(float(q[i+1]))
                    more = 2
                except: pass
        if c: break
    return more, c

In [96]:
def run(q, boost=[], b=1, n=10, fuzz=3):
    qcheck = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", q.lower()).strip().split()
    grammage = wt(qcheck)
    op, price = cost(qcheck)

    q += 4*int(b)*(' ' + ' '.join(boost))
    scores = run_sif(preproc(q), dlist, freqs=frequencies, model=model, fuzz=fuzz)
    df = t.copy()
    df["scores"] = scores
    print((scores > 1.005).sum())

    #price
    if price:
        if op==1:
            df.loc[df["Final Price"] < price, "scores"] += 0.005
        elif op==2:
            df.loc[df["Final Price"].between(price-10, price+10), "scores"] += 0.005
        elif op==3:
            df.loc[df["Final Price"] > price, "scores"] += 0.005

    #grammage
    if grammage:
        df.loc[df["wt"] == grammage, "scores"] += 0.005

    print((df["scores"]).mean())
    return df

In [98]:
df=run("sopa 270 rs 100 gm", boost=[], fuzz=6)
df.sort_values("scores", ascending=False)[["Product Description", "Grammage", "Final Price", "scores"]].head(10)

['sopa']
['pa', 'so', 'spa', 'soya', 'soap', 'stop']
0
0.46958511133042413


,Product Description,Grammage,Final Price,scores
2514,FUARE 125M PBT SUN MOIST SUN LOTION SPF50 PA U...,NaN,270,1.002879
3642,HGXWT 100G PTB SAFESUN UV SCREEN MATTE GEL PA ...,100G,18,1.002766
5606,WRMFP 60M PTB HRBL SUNCTRL DAILYMOISTURISER SP...,NaN,278,1.002293
2383,FEBXH 100 GM PTB DE TAN SUN SCREEN GEL CREAM S...,100 GM,811,1.001754
4593,ROYDQ 100G PTB SUNCARE SPF30 UVA PA+++UVB ALOE...,100G,437,1.001264
6115,FEBXH 100G PTB MOISTUR SUN BLOCK CREAM SPF 25 ...,100G,85,1.001141
7484,AQGXS 100 GM PLBOT SOYA SAUCE (B/W-,100 GM,277,0.999917
6125,OPGHO 50ML PLTUB SUNBLOCK LOTION PA++ SPF 30,50ML,78,0.997501
1272,FEBXH 40ML CDBOX REFRESHING SUN BLOCK SUN DEFE...,40ML,336,0.997219
3488,OXMWA 50 GM CDBOX SUNSCREEN GEL PA++ SPF 26,50 GM,86,0.997005


In [65]:
%timeit run("powder tomat 250 gm 150 rs", boost=["ayghd"])

['powder', 'tomat', 'ayghd', 'ayghd', 'ayghd', 'ayghd']
['powder', 'tomato', 'mat', 'to', 'ayghd', 'ayghd', 'ayghd', 'ayghd']
['powder', 'tomat', 'ayghd', 'ayghd', 'ayghd', 'ayghd']
['powder', 'tomato', 'mat', 'to', 'ayghd', 'ayghd', 'ayghd', 'ayghd']
['powder', 'tomat', 'ayghd', 'ayghd', 'ayghd', 'ayghd']
['powder', 'tomato', 'mat', 'to', 'ayghd', 'ayghd', 'ayghd', 'ayghd']
['powder', 'tomat', 'ayghd', 'ayghd', 'ayghd', 'ayghd']
['powder', 'tomato', 'mat', 'to', 'ayghd', 'ayghd', 'ayghd', 'ayghd']
1 loop, best of 3: 792 ms per loop
